In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2760)

In [15]:
x[:,:2]

matrix([[  972338.98097009,   969522.26503497],
        [ 1001259.35903116,   991961.28059715],
        [ 1855773.71165453,  1833622.70026465],
        [ 1905585.92669338,  1890611.50609783],
        [  458369.97103057,   464647.87886259],
        [  588645.39320544,   581513.65021207],
        [ 1191830.07697694,  1178897.98850843],
        [ 1190760.3098959 ,  1176936.88539783],
        [ 1137386.33974524,  1113887.70056067],
        [ 1130707.49420183,  1127714.3286336 ],
        [ 1113389.25366836,  1113762.18416443],
        [ 1084414.20749034,  1136162.95743499],
        [  269402.79473094,   268395.21414636],
        [  284683.0178454 ,   276706.1309289 ],
        [  463673.30885403,   444264.02389411],
        [  399206.80037492,   423910.59293823],
        [  635204.14879191,   623364.35465976],
        [  609048.58234714,   613484.75899901],
        [  974528.67626838,   989163.4304645 ],
        [  866425.74025559,   878289.14218884],
        [ 1482308.79640513,  1440510.833

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [8e-03, 8e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -7.98995316e+06  0.00000000e+00  5.86e+03 1.34e+03  1.00e+06     0s
   1  -6.29136555e+06 -4.13531916e+06  1.50e+03 3.44e+02  3.02e+05     0s
   2  -5.77535459e+06 -7.78348930e+06  1.50e+02 3.43e+01  7.28e+04     0s
   3  -6.17370749e+06 -6.62581023e+06  5.76e-01 1.32e-01  8.2

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[2.7762770534018084e-08,
 375017.4906433238,
 426740.493336354,
 9.50982987147353e-05,
 171873.59939529412,
 0.00016735303298123436,
 0.00016860311341113793,
 1.0712065540102888e-07,
 1.7180801155092626e-09,
 430474.15265008446,
 1.7180490707052365e-09,
 1.7180801280155981e-09,
 1.718062458924068e-09,
 1.7296437072151559e-09,
 262183.475691299,
 3.690276090124938e-09,
 1.997134437466423e-06,
 9.506473683084868e-05,
 212457.63407708541,
 447196.07027748716,
 0.00015787493614080952,
 310788.6306413933,
 449841.6855059123,
 4.255303062411313e-06,
 76583.22639999722,
 3834.6715428706,
 32273.93521316447,
 447950.5485432352,
 755071.1425958002,
 3.690276062357451e-09,
 3.975518052826035e-06,
 165875.46422458827,
 253942.3087651609,
 5.48272227682964e-07,
 253376.75405503053,
 668822.057097598,
 3.690276064750372e-09,
 4.694093873554837e-06,
 8.636477612828644e-06,
 328563.7087517183,
 623601.5070927081,
 0.00205624409723143,
 4.833131430001308e-06,
 3.6874605325277077e-09,
 3.47332250441508